<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qtd8elnr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qtd8elnr
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802823 sha256=2c187303f476bc10fef81ba6ecb33b976289371c80edaf710925603734a48fdb
  Stored in directory: /tmp/pip-ephem-wheel-cache-0bw7l03p/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [18]:
import whisper
import json, re

In [4]:
model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 116MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [9]:
result = model.transcribe("./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.mp4", verbose=True, fp16=False, language="ja") #ファイル指定

[00:00.000 --> 00:10.000] どうもこんにちは
[00:10.000 --> 00:12.000] ありがとうございます
[00:12.000 --> 00:19.000] ごめん 記者の時間いただきましてどうもありがとうございます
[00:20.000 --> 00:22.000] どうもいい
[00:24.000 --> 00:29.000] では早速質問をさせていただきたいと思います
[00:29.000 --> 00:31.000] ちょっとだけお願いします
[00:31.000 --> 00:49.000] では早速なんですけれども
[00:49.000 --> 00:55.000] まずフリーをどうでしたでしょうか
[00:56.000 --> 01:02.000] 今ちょっと回線がフリーズして聞こえなかったんですけど
[01:02.000 --> 01:07.000] フリーを導入されたのはいつ頃でしたでしょうか
[01:07.000 --> 01:19.000] 今から6年前で2018年の4月ですね
[01:19.000 --> 01:23.000] すごい そんな前からだったんですね
[01:23.000 --> 01:25.000] はい
[01:25.000 --> 01:31.000] その当時他の例えばクラウド会計のものとかって
[01:31.000 --> 01:33.000] ご検討されましたでしょうか
[01:33.000 --> 01:41.000] マネーフォアードとフリーとどっちにしようかなという話はちょっと
[01:41.000 --> 01:44.000] 私の中だけでありましたけども
[01:45.000 --> 01:53.000] フリーの方がどうも私はその会計に詳しい人間ではないので
[01:53.000 --> 02:03.000] 初心者向きだという話がちょっとしまったのレビューでありましたので
[02:03.000 --> 02:08.000] じゃあフリーの方いいかなと思ってフリーしました
[02:09.000 --> 02:11.000] 少し出しました
[02:11.000 --> 02:18.000] 価格とかはその時って検討されましたか企画というか
[02:18.000 --> 

In [28]:
segments = result['segments']
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
segment = []
for seg in segments:
  count = len(prog.findall(seg['text']))
  if count > 0: print(seg['text'])
  _seg = {'start': seg['start'], 'end': seg['end'], 'text': seg['text'], 'count': count}
  segment.append(_seg)

ちょっとだけお願いします
まずフリーをどうでしたでしょうか
フリーを導入されたのはいつ頃でしたでしょうか
ご検討されましたでしょうか
価格とかはその時って検討されましたか企画というか
2万弱というのは月月月月ですか
ご自身でやっていらっしゃるビジネスですかね
キャンセルした方がよろしいでしょうか
そうですか
